Get prediction metrics, confusion matrix, auc and f1 plot

In [2]:
from ultralytics import YOLO

model = YOLO('weights/v3_best.pt')

# 1) Validate on the 'val' split from your dataset YAML
metrics = model.val(
    data="../dataset/merged_currated/data.yaml",  
    split="val",  # we evaluate with the validation set        
    imgsz=1280,   # higher imgsz yields better result for fine grained / smaller objects
    batch=16,     # batch size depends on your gpu, set to larger for gpu with more vram 
    conf=0.60,    # the min confidence rate for the results, you can experiment with different values     
    device=0,     # set to 'cpu' if using cpu         
    plots=True,   # save PR curves, confusion matrix, etc.
    half=False    # use FP16 quantization, disable FP16 if it causes issues
)

Ultralytics 8.3.223  Python-3.9.24 torch-2.8.0+cu126 CPU (Intel Core i5-10400F 2.90GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 548.4194.7 MB/s, size: 320.6 KB)
val: Scanning C:\Users\Qwerty\Desktop\ANPR_Malaysia\dataset\merged_currated\valid\labels.cache... 73 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 73/73  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 5/5 0.2it/s 20.0s6.0ss
                   all         73        114      0.905      0.877      0.931      0.546
Speed: 5.2ms preprocess, 238.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to C:\Users\Qwerty\runs\detect\val13


Visualize the predictions (Side by side)

In [ ]:
from ultralytics import YOLO
import cv2, os, numpy as np
import matplotlib.pyplot as plt

model = YOLO("weights/v3_best.pt") #change your desired weights here
images_dir = '../dataset/merged_currated/valid/images/'  #change your dataset here
labels_dir = '../dataset/merged_currated/valid/labels/'  #change your dataset here
conf, imgsz = 0.60, 1280
device = 0

results = model.predict(source=images_dir, conf=conf, imgsz=imgsz, stream=True, verbose=False, device=device)

for r in results:
    img = cv2.imread(r.path)
    if img is None:
        print(f"Can't read: {r.path}")
        continue
    h, w = img.shape[:2]

    gt = img.copy()
    label_path = os.path.join(labels_dir, os.path.splitext(os.path.basename(r.path))[0] + ".txt")
    if os.path.exists(label_path):
        for line in open(label_path):
            _, xc, yc, bw, bh = map(float, line.split())
            x1, y1 = int((xc-bw/2)*w), int((yc-bh/2)*h)
            x2, y2 = int((xc+bw/2)*w), int((yc+bh/2)*h)
            cv2.rectangle(gt, (x1,y1), (x2,y2), (0,255,0), 2)
    pred = r.plot()

    combined = np.hstack((gt, pred))
    plt.imshow(cv2.cvtColor(combined, cv2.COLOR_BGR2RGB))
    plt.title("Ground Truth (Left) | Prediction (Right)")
    plt.axis('off')
    plt.show()
